<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/Jane_street_ai%E6%AF%94%E8%B3%BD%E4%BB%A3%E7%A2%BCdeepseek%E7%89%88Gemini%E6%94%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# -*- coding: utf-8 -*-
"""
Jane Street市場數據預測 - 繁體中文版
結合深度學習與梯度提升樹的集成模型
"""

# 環境準備
!pip install pandas scikit-learn xgboost tensorflow

import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GaussianNoise, Concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os

# 數據預處理函數
def 數據預處理(訓練數據, 測試數據=None):
    """
    數據預處理流程:
    1. 中位數填充缺失值
    2. 標準化特徵
    3. 生成目標變量(action)
    """
    # 特徵列選取
    特徵列 = [f'feature_{i}' for i in range(130)]

    # 使用訓練數據的中位數填充缺失值
    訓練數據填補 = 訓練數據.fillna(訓練數據.median())

    if 測試數據 is not None:
        測試數據填補 = 測試數據.fillna(訓練數據.median())  # 測試集使用訓練集統計量

    # 標準化處理（僅擬合訓練集）
    標準化器 = StandardScaler()
    訓練數據填補[特徵列] = 標準化器.fit_transform(訓練數據填補[特徵列])

    if 測試數據 is not None:
        測試數據填補[特徵列] = 標準化器.transform(測試數據填補[特徵列])

    # 目標變量生成（action=1當resp>0）
    訓練數據填補['action'] = (訓練數據填補['resp'] > 0).astype(int)

    if 測試數據 is not None:
        測試數據填補['action'] = (測試數據填補['resp'] > 0).astype(int)

    return 訓練數據填補, 測試數據填補, 特徵列

# 建立自編碼器+MLP模型
def 建立自編碼器_MLP(輸入維度):
    """
    建立多任務學習模型:
    1. 自編碼器部分用於特徵提取
    2. MLP部分用於分類預測
    """
    # 輸入層
    輸入層 = Input(shape=(輸入維度,))

    # 自編碼器部分（加入高斯噪聲防過擬合）
    x = GaussianNoise(0.15)(輸入層)
    x = Dense(128, activation='swish', kernel_initializer='he_normal')(x)
    x = Dense(64, activation='swish')(x)
    編碼層 = Dense(32, activation='swish', name='編碼輸出層')(x)
    解碼層 = Dense(輸入維度, activation='linear')(編碼層)

    # MLP分類部分（融合原始特徵與編碼特徵）
    融合層 = Concatenate()([輸入層, 編碼層])
    x = Dense(256, activation='swish')(融合層)
    x = Dense(128, activation='swish')(x)
    分類輸出 = Dense(1, activation='sigmoid')(x)

    # 多任務模型定義
    模型 = Model(inputs=輸入層, outputs=[解碼層, 分類輸出])
    模型.compile(
        loss={'dense_5': 'mse', 'dense_7': 'binary_crossentropy'},  # 需根據層名稱調整
        loss_weights=[0.2, 0.8],  # 強化分類任務權重
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
    )

    return 模型

# 主程序流程
if __name__ == "__main__":
    # 使用os.getcwd()獲取當前工作目錄，因為__file__在Jupyter Notebook中未定義
    腳本目錄 = os.getcwd()

    # 構建數據文件路徑
    訓練數據路徑 = os.path.join(腳本目錄, 'train.csv')
    測試數據路徑 = os.path.join(腳本目錄, 'test.csv')

    # 載入訓練數據(train_df)和測試數據(test_df)
    train_df = pd.read_csv(訓練數據路徑)
    test_df = pd.read_csv(測試數據路徑)

    # 數據預處理
    訓練數據處理後, 測試數據處理後, 特徵列 = 數據預處理(train_df, test_df)

    # 時間序列交叉驗證
    tscv = TimeSeriesSplit(n_splits=5)
    測試預測列表 = []

    for fold, (訓練索引, 驗證索引) in enumerate(tscv.split(訓練數據處理後)):
        print(f'正在訓練第 {fold+1} 折')

        # 數據分割
        X訓練 = 訓練數據處理後.iloc[訓練索引][特徵列]
        y訓練 = 訓練數據處理後.iloc[訓練索引]['action']
        X驗證 = 訓練數據處理後.iloc[驗證索引][特徵列]
        y驗證 = 訓練數據處理後.iloc[驗證索引]['action']

        # 訓練自編碼器（添加Early Stopping）
        自編碼器模型 = 建立自編碼器_MLP(len(特徵列))
        訓練歷史 = 自編碼器模型.fit(
            X訓練, [X訓練, y訓練],
            validation_data=(X驗證, [X驗證, y驗證]),
            epochs=50,
            batch_size=2048,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)],
            verbose=1
        )

        # 提取編碼特徵（通過層名稱獲取）
        編碼器 = Model(inputs=自編碼器模型.input, outputs=自編碼器模型.get_layer('編碼輸出層').output)
        訓練編碼特徵 = 編碼器.predict(X訓練)
        驗證編碼特徵 = 編碼器.predict(X驗證)

        # 合併特徵（水平堆疊）
        X訓練合併 = np.hstack([X訓練, 訓練編碼特徵])
        X驗證合併 = np.hstack([X驗證, 驗證編碼特徵])

        # 訓練XGBoost（啟用GPU加速）
        xgb模型 = XGBClassifier(
            n_estimators=800,
            max_depth=7,
            learning_rate=0.03,
            subsample=0.85,
            tree_method='gpu_hist',  # GPU加速
            eval_metric='logloss',
            use_label_encoder=False
        )
        xgb模型.fit(X訓練合併, y訓練)

        # 預測測試集
        測試編碼特徵 = 編碼器.predict(測試數據處理後[特徵列])
        測試合併特徵 = np.hstack([測試數據處理後[特徵列], 測試編碼特徵])
        測試預測列表.append(xgb模型.predict_proba(測試合併特徵)[:, 1])

    # 生成提交文件（閾值可調）
    最終預測 = np.mean(測試預測列表, axis=0)
    提交文件 = 測試數據處理後[['row_id']].copy()
    提交文件['action'] = (最終預測 > 0.5).astype(int)
    提交文件.to_csv('submission_v2.csv', index=False)

    # 效能監控
    # 1. 繪製訓練曲線
    plt.plot(訓練歷史.history['dense_7_loss'], label='訓練損失')  # 分類任務損失
    plt.plot(訓練歷史.history['val_dense_7_loss'], label='驗證損失')
    plt.legend()
    plt.title('模型訓練曲線')
    plt.show()

    # 2. 顯示XGBoost特徵重要性
    from xgboost import plot_importance
    plot_importance(xgb模型, max_num_features=15)
    plt.title('特徵重要性')
    plt.show()

    # 3. 預測分佈分析
    import seaborn as sns
    sns.histplot(最終預測, kde=True)
    plt.xlabel('預測概率')
    plt.title('測試集預測分佈')
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'